In [2]:
import json
from typing import Any

import pandas as pd
import seaborn as sns
from datasets import Dataset, DatasetDict, load_dataset
from langchain_core.utils.json import parse_json_markdown
from tqdm.auto import tqdm

sns.set_theme()

In [3]:
ds = load_dataset(
    "data/datasets/pl/swiss_franc_loans_source",
    data_files={"train": "train.json", "test": "test.json", "annotated": "annotated.json"},
)
ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating annotated split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'output'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['context', 'output'],
        num_rows: 1396
    })
    annotated: Dataset({
        features: ['context', 'output'],
        num_rows: 690
    })
})

# Validate `train`

In [15]:
train_df = ds["train"].to_pandas()
test_df = ds["annotated"].to_pandas()
test_df.head()

,context,output
0,Sygn. akt XV C 1471/12\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Nie"",""apelacja"":null,""..."
1,Sygn. akt V Ca 2924/12\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":""warsz..."
2,Sygn. akt I C 376/13\n\nWYROK\nW IMIENIU RZECZ...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":null,""..."
3,Sygn. akt I ACa 801/13\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":""Biały..."
4,Sygn. akt I ACa 812/13\n\nWYROK\nW IMIENIU RZE...,"{""sprawa_frankowiczow"":""Tak"",""apelacja"":""Gdańs..."


In [20]:
assert train_df.merge(test_df, on="context", how="left")["output_y"].notna().sum() == 0

In [24]:
train_df["context"].duplicated().sum()

np.int64(0)

In [21]:
out = pd.DataFrame(train_df["output"].apply(parse_json_markdown).to_list())
out.head()

,sprawa_frankowiczow,apelacja,data_wyroku,typ_sadu,instancja_sadu,podstawa_prawna,podstawa_prawna_podana,rodzaj_roszczenia,modyfikacje_powodztwa,typ_modyfikacji,...,odsetki_ustawowe,data_rozpoczecia_odsetek,koszty_postepowania,beneficjent_kosztow,zabezpieczenie_udzielone,rodzaj_zabezpieczenia,zabezpieczenie_pierwsza_instancja,czas_trwania_sprawy,wynik_sprawy,szczegoly_wyniku_sprawy
0,Tak,None,2010-12-14,Sąd Ochrony Konkurencji i Konsumentów,Sąd I instancji,"art. 3851 § 1 k.c., art. 3853 pkt 9 i 10 k.c.,...",Tak,Dotyczy wzorca umownego,None,None,...,Nie,None,Tak,Prezes Urzędu Ochrony Konkurencji i Konsumentów,None,None,None,None,Wygrana kredytobiorcy,Sąd uznał za niedozwolone i zakazał stosowania...
1,Nie,None,2010-12-21,Sąd Rejonowy,Sąd I instancji,art. 410 kc,Tak,O zapłatę,None,None,...,Nie,None,Tak,Bank (...) Spółka Akcyjna z siedzibą w W.,None,None,None,None,Wygrana banku,Sąd oddalił powództwo o zwrot prowizji za wcze...
2,Nie,None,2010-12-27,Sąd Ochrony Konkurencji i Konsumentów,Sąd I instancji,"art. 479(36) i następne k.p.c., art. 3851 § 1 ...",Tak,Dotyczy wzorca umownego,None,None,...,None,None,Tak,Powód,None,None,None,None,Wygrana kredytobiorcy,Sąd uznał postanowienie wzorca umowy za niedoz...
3,Tak,VIACa 635/11,2011-11-17,Sąd Ochrony Konkurencji i Konsumentów,Sąd odwoławczy,"art. 479(36) i następne k.p.c., art. 3851 § 1 ...",Tak,Dotyczy wzorca umownego,Nie,None,...,Nie,None,Tak,Powód,Nie,None,None,od 2009 do 2011,Wygrana kredytobiorcy,Sąd uznał postanowienie wzorca umowy o kredyt ...
4,Badanie wzorca umownego,None,2010-12-27,Sąd Ochrony Konkurencji i Konsumentów,Sąd I instancji,"art. 479(36) i następne k.p.c., art. 3851 § 1 ...",Tak,Dotyczy wzorca umownego,None,None,...,None,None,Tak,Powód,None,None,None,None,Wygrana kredytobiorcy,Sąd uznał postanowienie wzorca umowy za niedoz...


In [23]:
out.isna().mean(axis=0)

sprawa_frankowiczow                  0.0000
apelacja                             0.6930
data_wyroku                          0.0612
typ_sadu                             0.0064
instancja_sadu                       0.0144
podstawa_prawna                      0.1288
podstawa_prawna_podana               0.0992
rodzaj_roszczenia                    0.0216
modyfikacje_powodztwa                0.1204
typ_modyfikacji                      0.5820
status_kredytobiorcy                 0.1420
wspoluczestnictwo_powodowe           0.0290
wspoluczestnictwo_pozwanego          0.0330
typ_wspoluczestnictwa                0.4534
strony_umowy                         0.1066
wczesniejsze_skargi_do_rzecznika     0.3228
umowa_kredytowa                      0.1472
klauzula_niedozwolona                0.1106
wpisana_do_rejestru_uokik            0.2962
waluta_splaty                        0.1148
aneks_do_umowy                       0.1012
data_aneksu                          0.4754
przedmiot_aneksu                

# Analyse

In [25]:
def get_test_items_matching_annotated(dataset: DatasetDict) -> Dataset:
    def get_matching_test_item(annotated_item: dict[str, Any]) -> dict[str, Any]:
        found_items = []
        for test_item in dataset["test"]:
            if annotated_item["context"] == test_item["context"]:
                found_items.append(test_item)
        if not len(found_items) == 1:
            raise ValueError(f"Found {len(found_items)} items")
        return found_items[0]

    test_items_matching_annotated = []
    for annotated_item in tqdm(dataset["annotated"], "Matching test and annotated items"):
        found_item = get_matching_test_item(annotated_item)
        assert found_item["context"] == annotated_item["context"]
        test_items_matching_annotated.append(found_item)
        
    return Dataset.from_pandas(pd.DataFrame(test_items_matching_annotated))

In [26]:
get_test_items_matching_annotated(ds)

Matching test and annotated items:   0%|          | 0/690 [00:00<?, ?it/s]

Dataset({
    features: ['context', 'output'],
    num_rows: 690
})